# Causal Bayesian Network: 
## Causal model for tropical damage prediction in the Philippines

In [7]:
# Libraries
## dataframe and numerical libraries
import pandas as pd
import numpy as np


## visualization libraries
from plotnine import ggplot, aes, geom_line, geom_point

## ML and graphical models libraries (we will see which is reacher to use)
import pgmpy
import bayespy

### General ML library
import sklearn

INFO:matplotlib.font_manager:Failed to extract font properties from /System/Library/Fonts/Supplemental/NISC18030.ttf: In FT2Font: Could not set the fontsize (invalid pixel size; error code 0x17)
INFO:matplotlib.font_manager:Failed to extract font properties from /System/Library/Fonts/LastResort.otf: tuple indices must be integers or slices, not str
INFO:matplotlib.font_manager:Failed to extract font properties from /System/Library/Fonts/Apple Color Emoji.ttc: In FT2Font: Could not set the fontsize (invalid pixel size; error code 0x17)
INFO:matplotlib.font_manager:generated new fontManager


## The data

Observational data from the Philippines representing the building damage in the municipalities [number of municipalities...] caused
by several tropical cyclones observed between [yr1 to yr2]. The data includes hazard variables, physical vulnerability variables, and social vulnerability.

In [32]:
# importing the datasets

## base data without income classes
path_base_data = r'/Users/masinde/Projects/causal-fairness-Philippines-drrm/data/combined_input_data.csv'

ph_base_df = pd.read_csv(path_base_data, header = 0)


## base data without income classes
path_inc_data = r'/Users/masinde/Projects/causal-fairness-Philippines-drrm/data/ph_income_cls.csv'

ph_inc_df = pd.read_csv(path_inc_data, header = 0)

## base data merged with income classes
#path_base_inc= r'/Users/masinde/Projects/causal-fairness-Philippines-drrm/data/ph_base_inc_cls.csv'

#ph_base_inc_df = pd.read_csv(path_base_inc, header = 0)

In [25]:
# view first few row of base data
ph_base_df.head()

,Mun_Code,typhoon,HAZ_rainfall_Total,HAZ_rainfall_max_6h,HAZ_rainfall_max_24h,HAZ_v_max,HAZ_dis_track_min,GEN_landslide_per,GEN_stormsurge_per,GEN_Bu_p_inSSA,...,VUL_StrongRoof_SalvageWall,VUL_LightRoof_StrongWall,VUL_LightRoof_LightWall,VUL_LightRoof_SalvageWall,VUL_SalvagedRoof_StrongWall,VUL_SalvagedRoof_LightWall,VUL_SalvagedRoof_SalvageWall,VUL_vulnerable_groups,VUL_pantawid_pamilya_beneficiary,DAM_perc_dmg
0,PH175101000,durian2006,185.828571,14.716071,7.381696,55.032241,2.478142,2.64,6.18,6.18,...,0.097425,2.533055,41.892832,1.002088,0.000000,0.027836,0.083507,2.951511,46.931106,3.632568
1,PH030801000,durian2006,28.487500,1.893750,1.070833,23.402905,136.527982,0.78,40.87,40.80,...,0.118842,0.248487,2.182368,0.000000,0.000000,0.010804,0.010804,0.867603,8.967156,NaN
2,PH083701000,durian2006,8.818750,0.455208,0.255319,8.728380,288.358553,0.06,0.00,0.00,...,0.850008,1.218595,13.645253,0.549120,0.030089,0.090266,0.112833,3.338873,25.989168,NaN
3,PH045601000,durian2006,193.083333,13.283333,7.378472,52.891105,41.386956,3.53,2.77,2.77,...,0.697085,1.774398,15.082383,0.443599,0.000000,0.000000,0.095057,1.953155,18.821293,NaN
4,PH015501000,durian2006,24.175000,2.408333,0.957639,10.945624,274.953818,1.52,1.28,1.28,...,0.197179,0.667374,15.592295,0.075838,0.000000,0.015168,0.075838,2.131755,32.185651,NaN


In [27]:
# view first few rows of income class data
ph_inc_df.head()

,Unnamed: 0,Municipality,10 Digit Code,Correspondence Code,Income Class,Population(2020 Census)
0,0,Pateros,1381701000,137606000.0,1st,"65,227"
1,1,Bangued,1400101000,140101000.0,1st,"50,382"
2,2,Boliney,1400102000,140102000.0,5th,"4,551"
3,3,Bucay,1400103000,140103000.0,5th,"17,953"
4,4,Bucloc,1400104000,140104000.0,6th,"2,395"


In [34]:
# view first few rows of combined data 
ph_base_inc_df.head()

,Unnamed: 0,Municipality,10 Digit Code,Correspondence Code,Income Class,Population(2020 Census)
0,0,Pateros,1381701000,137606000.0,1st,"65,227"
1,1,Bangued,1400101000,140101000.0,1st,"50,382"
2,2,Boliney,1400102000,140102000.0,5th,"4,551"
3,3,Bucay,1400103000,140103000.0,5th,"17,953"
4,4,Bucloc,1400104000,140104000.0,6th,"2,395"
